<a href="https://colab.research.google.com/github/oliverfoster27/Practical-Machine-Learning/blob/master/Week%207/C7_Exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.layers import Dropout


import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout, BatchNormalization
from keras.utils import to_categorical
import keras.backend as K
from keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import mnist
from keras.optimizers import Adam

In [30]:
(X_train, y_train), (X_test, y_test) = mnist.load_data('/tmp/mnist.npz')
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)
X_train.shape, X_test.shape, y_train, y_test

((60000, 784),
 (10000, 784),
 array([5, 0, 4, ..., 5, 6, 8], dtype=uint8),
 array([7, 2, 1, ..., 4, 5, 6], dtype=uint8))

In [0]:
X_train = X_train / 255
X_test = X_test / 255

# Exercise
Make a dense neural network with 95%+ accuracy on Mnist that has the smallest number of neurons possible by experimenting with Dropout, and Batch Norm

In [0]:
learning_grid = [10e-4, 10e-5, 10e-6, 10e-7, 10e-8]
input_dense_grid = np.linspace(10, 100, 19).astype(int)
p_grid = [0.2, 0.4, 0.6, 0.8]

In [0]:
from keras.models import Sequential

def find_minimal_network(features, output):

  data = []

  for input_dense in input_dense_grid:

    for p in p_grid:

      for learning_rate in learning_grid:

        K.clear_session()

        model = Sequential()

        model.add(Dense(input_dense, activation='relu', input_shape=(784,)))
        model.add(Dropout(p))
        model.add(BatchNormalization())
        model.add(Dense(10, activation='softmax'))

        callback_list = [EarlyStopping(monitor='val_acc', mode='max',
                                       verbose=1, patience=10)]

        optimizer = Adam(lr=learning_rate)
        model.compile(optimizer=optimizer,
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])

        print(model.summary())

        h = model.fit(features, output, epochs=500, validation_split=0.3,
                     callbacks=callback_list)

        if h.history['acc'][-1] >= 0.95:
          res={
            'Input Dense Layer Size': input_dense,
            'Learning Rate': learning_rate,
            'Dropout Proportion': p,
            'Validation Accuracy': h.history['acc'][-1]
          }
          # If we've satisfied 95% accuracy leave the learning rate grid
          return res

In [66]:
find_minimal_network(X_train, y_train)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                7850      
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 10)                40        
_________________________________________________________________
dense_2 (Dense)              (None, 10)                110       
Total params: 8,000
Trainable params: 7,980
Non-trainable params: 20
_________________________________________________________________
None
Train on 42000 samples, validate on 18000 samples
Epoch 1/500
42000/42000 [==============================] - 3s 82us/step - loss: 2.6639 - acc: 0.0968 - val_loss: 2.5783 - val_acc: 0.0961
Epoch 2/500
42000/42000 [==============================] - 3s 67us/step - loss

KeyboardInterrupt: ignored